<a href="https://colab.research.google.com/github/alermar69/glebmikha/blob/master/coffee_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz
!tar xzf sqlite.tar.gz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.6/pysqlite3/_sqlite3.cpython-36m-x86_64-linux-gnu.so \
    /usr/lib/python3.6/lib-dynload/_sqlite3.cpython-36m-x86_64-linux-gnu.so
# then MENU: Runtime > Restart runtime ...

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
sqlite3.sqlite_version

'3.34.0'

In [ ]:
print('''
Есть таблица с данными по продажам кофе:
SALE_DTTM – точное время продажи,
CARD_NUMBER – номер скидочной карты
COFFEE_TYPE – название проданного кофейного напитка
PRICE – полная стоимость
DISCOUNT – размер скидки в %
GROCERY_FLG – флаг покупки выпечки в том же чеке
Составьте алгоритм SQL-запросов для решения следующих заданий:
Вычислить медианное значение суммарных месячных трат постоянных клиентов за каждый месяц 
(при отсутствии встроенной функции, вычисляющей медиану).

Вычислить максимальное количество последовательных дней, когда один и тот же клиент покупал кофе с выпечкой.

В кофейне есть акция – при ежедневной покупке кофе с выпечкой можно купить термокружку со скидкой. 
При этом размер скидки на термокружку получается путём последовательного применения всех скидок на кофе 
с выпечкой, применённых за накопленную на момент покупки термокружки серию ежедневных покупок кофе с 
выпечкой.Вычислить максимальную скидку на термокружку, которую мог достичь клиент (размер скидки клиента
 не является константой).
''')


Есть таблица с данными по продажам кофе:
SALE_DTTM – точное время продажи,
CARD_NUMBER – номер скидочной карты
COFFEE_TYPE – название проданного кофейного напитка
PRICE – полная стоимость
DISCOUNT – размер скидки в %
GROCERY_FLG – флаг покупки выпечки в том же чеке
Составьте алгоритм SQL-запросов для решения следующих заданий:
Вычислить медианное значение суммарных месячных трат постоянных клиентов за каждый месяц 
(при отсутствии встроенной функции, вычисляющей медиану).

Вычислить максимальное количество последовательных дней, когда один и тот же клиент покупал кофе с выпечкой.

В кофейне есть акция – при ежедневной покупке кофе с выпечкой можно купить термокружку со скидкой. 
При этом размер скидки на термокружку получается путём последовательного применения всех скидок на кофе 
с выпечкой, применённых за накопленную на момент покупки термокружки серию ежедневных покупок кофе с 
выпечкой.Вычислить максимальную скидку на термокружку, которую мог достичь клиент (размер скидки клиента

In [ ]:
1000 * (1-0.05)*(1-0.1)*((1-0.05))

812.25

# 0. Генерим данные

Я зафиксил рандом стейт и все пересчитал, так что цифры теперь отличаются от видоса. Но смысл остается тем же

In [ ]:
dt = pd.DataFrame({'dt':pd.date_range('2020-04-01','2020-05-31')})

In [ ]:
np.random.seed(42)
dt['num'] = np.random.poisson(30,size=len(dt))

In [ ]:
dt

,dt,num
0,2020-04-01,28
1,2020-04-02,34
2,2020-04-03,23
3,2020-04-04,32
4,2020-04-05,36
...,...,...
56,2020-05-27,30
57,2020-05-28,26
58,2020-05-29,32
59,2020-05-30,18


In [ ]:
df = pd.DataFrame({'sale_dttm':(dt['dt'].apply(lambda x: [x]) * dt['num']).sum()})

In [ ]:
df

,sale_dttm
0,2020-04-01
1,2020-04-01
2,2020-04-01
3,2020-04-01
4,2020-04-01
...,...
1816,2020-05-31
1817,2020-05-31
1818,2020-05-31
1819,2020-05-31


In [ ]:
df['card_number'] = pd.Series(list(range(40))+[np.nan]*10).sample(len(df),replace=True,random_state=42).values

In [ ]:
df['coffee_type'] = pd.Series(['espresso','americano','cappuccino','cappuccino','cappuccino','latte']).sample(len(df),replace=True,random_state=42).values

In [ ]:
coffee_price = pd.DataFrame({'espresso':[100],'americano':[120],'cappuccino':[130],'latte':[150]}).T.reset_index()

In [ ]:
coffee_price

,index,0
0,espresso,100
1,americano,120
2,cappuccino,130
3,latte,150


In [ ]:
coffee_price.columns = ['coffee_type','coffee_price']

In [ ]:
df

,sale_dttm,card_number,coffee_type
0,2020-04-01,38.0,cappuccino
1,2020-04-01,28.0,cappuccino
2,2020-04-01,14.0,cappuccino
3,2020-04-01,NaN,cappuccino
4,2020-04-01,7.0,cappuccino
...,...,...,...
1816,2020-05-31,28.0,cappuccino
1817,2020-05-31,21.0,espresso
1818,2020-05-31,3.0,latte
1819,2020-05-31,19.0,cappuccino


In [ ]:
df = df.merge(coffee_price,how='left',on='coffee_type')

In [ ]:
df

,sale_dttm,card_number,coffee_type,coffee_price
0,2020-04-01,38.0,cappuccino,130
1,2020-04-01,28.0,cappuccino,130
2,2020-04-01,14.0,cappuccino,130
3,2020-04-01,NaN,cappuccino,130
4,2020-04-01,7.0,cappuccino,130
...,...,...,...,...
1816,2020-05-31,28.0,cappuccino,130
1817,2020-05-31,21.0,espresso,100
1818,2020-05-31,3.0,latte,150
1819,2020-05-31,19.0,cappuccino,130


In [ ]:
df['discount'] = pd.Series([0,0,0,0,0,0,0.05,0.05,0.05,0.05,0.1,0.15]).sample(len(df),replace=True,random_state=42).values

In [ ]:
df

,sale_dttm,card_number,coffee_type,coffee_price,discount
0,2020-04-01,38.0,cappuccino,130,0.05
1,2020-04-01,28.0,cappuccino,130,0.00
2,2020-04-01,14.0,cappuccino,130,0.10
3,2020-04-01,NaN,cappuccino,130,0.05
4,2020-04-01,7.0,cappuccino,130,0.00
...,...,...,...,...,...
1816,2020-05-31,28.0,cappuccino,130,0.00
1817,2020-05-31,21.0,espresso,100,0.00
1818,2020-05-31,3.0,latte,150,0.10
1819,2020-05-31,19.0,cappuccino,130,0.05


In [ ]:
df['grocery_flg'] = pd.Series([0,0,1,1]).sample(len(df),replace=True,random_state=42).values

In [ ]:
df

,sale_dttm,card_number,coffee_type,coffee_price,discount,grocery_flg
0,2020-04-01,38.0,cappuccino,130,0.05,1
1,2020-04-01,28.0,cappuccino,130,0.00,1
2,2020-04-01,14.0,cappuccino,130,0.10,0
3,2020-04-01,NaN,cappuccino,130,0.05,1
4,2020-04-01,7.0,cappuccino,130,0.00,1
...,...,...,...,...,...,...
1816,2020-05-31,28.0,cappuccino,130,0.00,0
1817,2020-05-31,21.0,espresso,100,0.00,1
1818,2020-05-31,3.0,latte,150,0.10,0
1819,2020-05-31,19.0,cappuccino,130,0.05,1


In [ ]:
df['grocery_flg'].mean()

0.49807797913234486

In [ ]:
df['price'] = (df['coffee_price'] + df['grocery_flg'] * 100)*(1-df['discount'])

In [ ]:
con = sqlite3.connect('db')

In [ ]:
cur = con.cursor()

In [ ]:
df.to_sql('coffee_sales',con,index=False,if_exists='replace')

In [ ]:
sql = '''select * from coffee_sales t limit 10'''

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
select(sql)

,sale_dttm,card_number,coffee_type,coffee_price,discount,grocery_flg,price
0,2020-04-01 00:00:00,38.0,cappuccino,130,0.05,1,218.5
1,2020-04-01 00:00:00,28.0,cappuccino,130,0.00,1,230.0
2,2020-04-01 00:00:00,14.0,cappuccino,130,0.10,0,117.0
3,2020-04-01 00:00:00,NaN,cappuccino,130,0.05,1,218.5
4,2020-04-01 00:00:00,7.0,cappuccino,130,0.00,1,230.0
5,2020-04-01 00:00:00,20.0,americano,120,0.05,1,209.0
6,2020-04-01 00:00:00,38.0,cappuccino,130,0.05,0,123.5
7,2020-04-01 00:00:00,18.0,cappuccino,130,0.00,0,130.0
8,2020-04-01 00:00:00,22.0,cappuccino,130,0.05,1,218.5
9,2020-04-01 00:00:00,10.0,cappuccino,130,0.10,0,117.0


# 1. Вычислить медианное значение суммарных месячных трат постоянных клиентов за каждый месяц (при отсутствии встроенной функции, вычисляющей медиану).

In [ ]:
sql = '''
select t.month, avg(t.price_sum) as mean_price,
median(t.price_sum) as median_price

from 

(select date(t.sale_dttm,'start of month') as month, 
t.card_number, 
sum(t.price) as price_sum from coffee_sales t
group by date(t.sale_dttm,'start of month'),t.card_number) t

group by t.month

'''

In [ ]:
df['month'] = df['sale_dttm'].values.astype('datetime64[M]')

In [ ]:
m = df.groupby(['month','card_number'])['price'].sum().reset_index()

In [ ]:
m.groupby('month')['price'].agg(['count','median'])

,count,median
month,,
2020-04-01,40,2960.00
2020-05-01,40,3209.25


In [ ]:
t = pd.DataFrame({'group':['a','a','a','b','b','b','b'],'val':[1,2,3,1,2,3,4]})

In [ ]:
t

,group,val
0,a,1
1,a,2
2,a,3
3,b,1
4,b,2
5,b,3
6,b,4


In [ ]:
t.groupby('group')['val'].agg(['count','median'])

,count,median
group,,
a,3,2.0
b,4,2.5


In [ ]:
t['rnk'] = t.groupby('group')['val'].rank()

In [ ]:
t

,group,val,rnk
0,a,1,1.0
1,a,2,2.0
2,a,3,3.0
3,b,1,1.0
4,b,2,2.0
5,b,3,3.0
6,b,4,4.0


In [ ]:
'''PERCENTILE_CONT(height, 0.5) OVER(PARTITION BY sex) AS median_height_per_sex'''

'PERCENTILE_CONT(height, 0.5) OVER(PARTITION BY sex) AS median_height_per_sex'

https://www.sqlservercentral.com/forums/topic/calculating-a-median-for-groups -- восьмая ссылка в гугле по запросу "sql median with group by" -- надо быть упорным и просматривать ссылки гугла, пока не найдешь то, что нужно. Не всегда оно выдается первой ссылкой.

In [ ]:
sql = '''

select 

 x.month,

       AVG(x.price_sum) AS Median


 from 

(select 

t.*,

ROW_NUMBER() OVER (PARTITION BY t.month ORDER BY t.price_sum)

               AS

                      RowNum,

               COUNT(*) OVER (PARTITION BY t.month)

               AS

                      RowCnt


from 

(select date(t.sale_dttm,'start of month') as month, 
t.card_number, 
sum(t.price) as price_sum from coffee_sales t

--where t.card_number is not null

group by date(t.sale_dttm,'start of month'),t.card_number) t) x

WHERE  x.RowNum IN ( ( x.RowCnt + 1 ) / 2, ( x.RowCnt + 2 ) / 2 )

GROUP  BY x.month



'''

In [ ]:
select(sql)

,month,Median
0,2020-04-01,2970.0
1,2020-05-01,3222.5


In [ ]:
m = df.groupby(['month','card_number'],dropna=False)['price'].sum().reset_index()

In [ ]:
m.groupby('month')['price'].agg(['count','median'])

,count,median
month,,
2020-04-01,41,2970.0
2020-05-01,41,3222.5


# 2. Вычислить максимальное количество последовательных дней, когда один и тот же клиент покупал кофе с выпечкой.


In [ ]:
sql = '''

drop table if exists sales_day_1;

create table sales_day_1 as

select date(t.sale_dttm) as dt,
t.card_number,
max(t.grocery_flg) as grocery_flg,
count(1) as num_purchases,

max(case when t.grocery_flg = 1 then t.discount else 0 end) as discount

 from coffee_sales t

group by date(t.sale_dttm),
t.card_number

'''

In [ ]:
cur.executescript(sql)

фишка подзапрос в питоне

In [ ]:
subq = '''select date(t.sale_dttm) as dt,
t.card_number,
max(t.grocery_flg) as grocery_flg,
count(1) as num_purchases,

-- add multipication https://stackoverflow.com/a/5416210
max(case when t.grocery_flg = 1 then t.discount else 0 end) as discount

 from coffee_sales t

group by date(t.sale_dttm),
t.card_number'''

In [ ]:
sql = f'''select * from ({subq}) t '''

In [ ]:
print(sql)

select * from (select date(t.sale_dttm) as dt,
t.card_number,
max(t.grocery_flg) as grocery_flg,
count(1) as num_purchases,

max(case when t.grocery_flg = 1 then t.discount else 0 end) as discount

 from coffee_sales t

group by date(t.sale_dttm),
t.card_number) t 


In [ ]:
select(sql)

,dt,card_number,grocery_flg,num_purchases,discount
0,2020-04-01,NaN,1,3,0.05
1,2020-04-01,1.0,1,2,0.00
2,2020-04-01,2.0,0,1,0.00
3,2020-04-01,7.0,1,1,0.00
4,2020-04-01,10.0,1,2,0.10
...,...,...,...,...,...
1147,2020-05-31,34.0,1,1,0.05
1148,2020-05-31,35.0,1,1,0.05
1149,2020-05-31,36.0,1,1,0.00
1150,2020-05-31,38.0,0,1,0.00


In [ ]:
sql = '''
drop table if exists sales_day_2;

create table sales_day_2 as


select t.*,

lag(t.dt) over (partition by t.card_number order by t.dt) as prev_dt,

julianday(t.dt) - julianday(lag(t.dt) over (partition by t.card_number order by t.dt)) as days_since_last_purchase,

lag(t.grocery_flg) over (partition by t.card_number order by t.dt) as prev_grocery_flg


 from sales_day_1  t
'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select * from sales_day_2 t 
where t.card_number = 39'''

In [ ]:
select(sql)

,dt,card_number,grocery_flg,num_purchases,discount,prev_dt,days_since_last_purchase,prev_grocery_flg
0,2020-04-01,39.0,1,1,0.00,None,NaN,NaN
1,2020-04-03,39.0,1,2,0.00,2020-04-01,2.0,1.0
2,2020-04-05,39.0,1,1,0.05,2020-04-03,2.0,1.0
3,2020-04-09,39.0,0,2,0.00,2020-04-05,4.0,1.0
4,2020-04-11,39.0,1,1,0.00,2020-04-09,2.0,0.0
5,2020-04-14,39.0,1,2,0.00,2020-04-11,3.0,1.0
6,2020-04-16,39.0,0,1,0.00,2020-04-14,2.0,1.0
7,2020-04-17,39.0,1,1,0.05,2020-04-16,1.0,0.0
8,2020-04-19,39.0,0,1,0.00,2020-04-17,2.0,1.0
9,2020-04-24,39.0,1,1,0.05,2020-04-19,5.0,0.0


В таком виде я до решения не додумался и решил сделать по классике, через темплейт где есть все даты на каждого клиента, как здесь в секции отток https://colab.research.google.com/drive/17Er-SMqRrJW7sDeKvEzyrgrGQJMmM8og

In [ ]:
sql = '''select distinct(t.card_number) as card_number from coffee_sales t
where t.card_number is not null'''

In [ ]:
select(sql)

,card_number
0,38.0
1,28.0
2,14.0
3,7.0
4,20.0
5,18.0
6,22.0
7,10.0
8,23.0
9,35.0


In [ ]:
min_date = '''select min(date(t.sale_dttm)) as dt from coffee_sales t'''
max_date = '''select max(date(t.sale_dttm)) as dt from coffee_sales t'''

In [ ]:
all_customers = '''select distinct(t.card_number) as card_number from coffee_sales t
where t.card_number is not null'''

In [ ]:
sql = f'''
drop table if exists day_template;
create table day_template as

WITH RECURSIVE dates(date) AS (
  VALUES(({min_date}))
  UNION ALL
  SELECT date(date, '+1 day')
  FROM dates
  WHERE date < ({max_date})
)
SELECT date as dt, card_number FROM dates
join ({all_customers}) c
'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select * from day_template t limit 10'''

In [ ]:
select(sql)

,dt,card_number
0,2020-04-01,38.0
1,2020-04-01,28.0
2,2020-04-01,14.0
3,2020-04-01,7.0
4,2020-04-01,20.0
5,2020-04-01,18.0
6,2020-04-01,22.0
7,2020-04-01,10.0
8,2020-04-01,23.0
9,2020-04-01,35.0


In [ ]:
sql = '''


drop table if exists sales_day_3;
create table sales_day_3 as

select t.*,

case when sd.grocery_flg > 0 then 1 else 0 end as grocery_flg,
sd.num_purchases,
sd.discount,

case when sd.num_purchases > 0 then 1 else 0 end as active,
lag(case when sd.num_purchases > 0 then 1 else 0 end) over (partition by t.card_number order by t.dt) as prev_active,


lag(case when sd.grocery_flg > 0 then 1 else 0 end) over (partition by t.card_number order by t.dt) as prev_grocery_flg


 from day_template t
left join sales_day_1 sd
on t.card_number = sd.card_number and 
t.dt = sd.dt



'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''


drop table if exists sales_day_4;
create table sales_day_4 as

select t.*,
case when t.active != t.prev_active then 1 else 0 end change_active,



SUM(case when t.active != t.prev_active then 1 else 0 end) OVER (
  partition by t.card_number
        ORDER BY t.dt 
        ROWS BETWEEN 
            UNBOUNDED PRECEDING 
            AND CURRENT ROW 
        ) as session_id,


SUM(case when t.grocery_flg != t.prev_grocery_flg then 1 else 0 end) OVER (
  partition by t.card_number
        ORDER BY t.dt 
        ROWS BETWEEN 
            UNBOUNDED PRECEDING 
            AND CURRENT ROW 
        ) as session_id_grocery


 from sales_day_3 t 

'''

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select t.session_id_grocery,
t.card_number,
max(grocery_flg) as grocery_flg,
count(1) as days_cnt

--,mul(1-t.discount) as final_discount

 from sales_day_4 t
 group by
 t.session_id_grocery,
t.card_number

having max(grocery_flg) = 1

order by count(1) desc
 
 '''

In [ ]:
select(sql)

,session_id_grocery,card_number,grocery_flg,days_cnt
0,1,31.0,1,6
1,17,33.0,1,5
2,2,38.0,1,4
3,19,2.0,1,4
4,20,28.0,1,4
...,...,...,...,...
481,29,16.0,1,1
482,29,24.0,1,1
483,30,23.0,1,1
484,31,16.0,1,1


In [ ]:
sql = '''select * from sales_day_4 t
where t.card_number = 31 and t.session_id_grocery = 1'''

In [ ]:
select(sql)

,dt,card_number,grocery_flg,num_purchases,discount,active,prev_active,prev_grocery_flg,change_active,session_id,session_id_grocery
0,2020-04-05,31.0,1,1,0.15,1,0,0,1,1,1
1,2020-04-06,31.0,1,2,0.15,1,1,1,0,1,1
2,2020-04-07,31.0,1,2,0.10,1,1,1,0,1,1
3,2020-04-08,31.0,1,3,0.00,1,1,1,0,1,1
4,2020-04-09,31.0,1,1,0.00,1,1,1,0,1,1
5,2020-04-10,31.0,1,1,0.05,1,1,1,0,1,1


In [ ]:
sql = '''select * from coffee_sales t
where t.card_number = 31
order by t.sale_dttm'''

In [ ]:
select(sql)

,sale_dttm,card_number,coffee_type,coffee_price,discount,grocery_flg,price
0,2020-04-05 00:00:00,31.0,cappuccino,130,0.15,1,195.5
1,2020-04-06 00:00:00,31.0,cappuccino,130,0.00,0,130.0
2,2020-04-06 00:00:00,31.0,latte,150,0.15,1,212.5
3,2020-04-07 00:00:00,31.0,cappuccino,130,0.05,1,218.5
4,2020-04-07 00:00:00,31.0,espresso,100,0.10,1,180.0
5,2020-04-08 00:00:00,31.0,cappuccino,130,0.00,1,230.0
6,2020-04-08 00:00:00,31.0,cappuccino,130,0.00,0,130.0
7,2020-04-08 00:00:00,31.0,latte,150,0.00,0,150.0
8,2020-04-09 00:00:00,31.0,cappuccino,130,0.00,1,230.0
9,2020-04-10 00:00:00,31.0,espresso,100,0.05,1,190.0


# 3. Серии вторым способом

In [ ]:
sql = '''

drop table if exists sales_day_1;

create table sales_day_1 as

select date(t.sale_dttm) as dt,
t.card_number,
max(t.grocery_flg) as grocery_flg,
count(1) as num_purchases,


-- add multipication https://stackoverflow.com/a/5416210
max(case when t.grocery_flg = 1 then t.discount else 0 end) as discount

 from coffee_sales t

where t.grocery_flg = 1

group by date(t.sale_dttm),
t.card_number

'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''
drop table if exists sales_day_2;

create table sales_day_2 as


select t.*,

lag(t.dt) over (partition by t.card_number order by t.dt) as prev_dt,

julianday(t.dt) - julianday(lag(t.dt) over (partition by t.card_number order by t.dt)) as days_since_last_purchase,

case when julianday(t.dt) - julianday(lag(t.dt) over (partition by t.card_number order by t.dt)) is null or
julianday(t.dt) - julianday(lag(t.dt) over (partition by t.card_number order by t.dt)) > 1 then 1 else 0 end as new_session_flag


from sales_day_1  t
'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''
drop table if exists sales_day_3;
create table sales_day_3 as

select t.*,

SUM(t.new_session_flag) OVER (
  partition by t.card_number
        ORDER BY t.dt 
        ROWS BETWEEN 
            UNBOUNDED PRECEDING 
            AND CURRENT ROW 
        ) as session_id


from sales_day_2 t


order by t.dt
'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select t.card_number, t.session_id, count(1) from sales_day_3 t

where t.card_number is not null

group by t.card_number, t.session_id
order by count(1) desc

'''

In [ ]:
select(sql)

,card_number,session_id,count(1)
0,31.0,1,6
1,33.0,9,5
2,2.0,10,4
3,28.0,11,4
4,38.0,2,4
...,...,...,...
481,39.0,7,1
482,39.0,8,1
483,39.0,9,1
484,39.0,10,1


# 4. Еще один способ серий

https://stackoverflow.com/questions/20402089/detect-consecutive-dates-ranges-using-sql

In [ ]:
sql = '''select * from sales_day_1 t limit 10'''

In [ ]:
select(sql)

,dt,card_number,grocery_flg,num_purchases,discount
0,2020-04-01,NaN,1,2,0.05
1,2020-04-01,1.0,1,1,0.00
2,2020-04-01,7.0,1,1,0.00
3,2020-04-01,10.0,1,1,0.10
4,2020-04-01,20.0,1,2,0.05
5,2020-04-01,21.0,1,1,0.00
6,2020-04-01,22.0,1,1,0.05
7,2020-04-01,23.0,1,3,0.05
8,2020-04-01,28.0,1,1,0.00
9,2020-04-01,32.0,1,1,0.15


In [ ]:
sql = '''WITH t AS (
  SELECT t.dt as d,
  t.card_number,
  ROW_NUMBER() OVER(partition by t.card_number ORDER BY t.dt) i
  FROM sales_day_1 t
  GROUP BY t.dt
)
SELECT MIN(d),MAX(d)
FROM t
GROUP BY DATEDIFF(day,i,d)'''

In [ ]:
select(sql)

Слишком геморно и я забил

# 5. Скидка

https://stackoverflow.com/a/5416210

In [ ]:
'''select EXP(SUM(LOG(yourColumn))) As ColumnProduct from yourTable'''

'select EXP(SUM(LOG(yourColumn))) As ColumnProduct from yourTable'

In [ ]:
np.log(np.e)

1.0

In [ ]:
con.create_function("log", 1, np.log)
con.create_function("exp", 1, np.exp)

In [ ]:
sql = '''select 

t.card_number,
t.session_id,

count(1) as purchase_count,


1-EXP(SUM(log(1-t.discount))) as final_discount


from sales_day_3 t

where t.card_number is not null

group by
t.card_number,
t.session_id

order by 1-EXP(SUM(log(1-t.discount))) desc

'''

In [ ]:
select(sql)

,card_number,session_id,purchase_count,final_discount
0,31.0,1,6,0.382263
1,11.0,5,3,0.313625
2,33.0,9,5,0.273250
3,2.0,10,4,0.268975
4,29.0,13,2,0.235000
...,...,...,...,...
481,39.0,2,1,0.000000
482,39.0,4,1,0.000000
483,39.0,5,1,0.000000
484,39.0,9,1,0.000000


In [ ]:
sql = '''select 

*

from sales_day_3 t

where t.card_number = 31 and t.session_id = 1

'''

In [ ]:
t = select(sql)

In [ ]:
1-(1-t['discount']).prod()

0.38226250000000006

# 6. Имеется следующая структура данных - данные записываются для каждого пользователя, совершавшего покупки, каждый день. Под каждую дату есть своя папка, внутри неё папки для каждого пользователя. Внутри каждой папки есть файл data.csv, где и хранятся данные. Требуется собрать данные из папки data в 1 датафрэйм, где будут две колонки из самих файлов (product_id, quantity), а также имя пользователя, и дата этих покупок (соответствует названию папки, где лежит папка с пользователем)

In [ ]:
! ls

build	       LICENSE	    README.md	 setup.py   sqlite3.h	   test
build-scripts  MANIFEST.in  sample_data  sqlite     sqlite.tar.gz  TODO
db	       pysqlite3    setup.cfg	 sqlite3.c  src


In [ ]:
! pwd

/content


In [ ]:
! mkdir mydir

In [ ]:
! touch ./mydir/file

In [ ]:
df = pd.DataFrame({'product_id':[1,2],'quantity':[2,3]})

In [ ]:
df.to_csv('./mydir/data.csv',index=False)

In [ ]:
! rm /content/mydir/data.csv

In [ ]:
! rm /content/mydir/file

In [ ]:
df = pd.DataFrame({'user_id':[1,1,2,2],
                   'dt':['2020-12-24','2020-12-23','2020-12-24','2020-12-23'],
                   'product_id':[1,2,3,2],
                   'quantity':[2,3,5,7]})

In [ ]:
df

,user_id,dt,product_id,quantity
0,1,2020-12-24,1,2
1,1,2020-12-23,2,3
2,2,2020-12-24,3,5
3,2,2020-12-23,2,7


In [ ]:
import os

In [ ]:
os.makedirs('/content/mydir/2020-12-24')

In [ ]:
df['dt'].unique()

array(['2020-12-24', '2020-12-23'], dtype=object)

In [ ]:
for dt in df['dt'].unique():

  date_dir = f'/content/mydir/{dt}'

  if not os.path.exists(date_dir):
    os.makedirs(date_dir)

  for user_id in df[df['dt']==dt]['user_id'].unique():

    user_dir = f'/content/mydir/{dt}/{user_id}'
    if not os.path.exists(user_dir):
      os.makedirs(user_dir)
      
    (df[(df['dt']==dt)&(df['user_id']==user_id)][['product_id','quantity']]
     .to_csv(user_dir+'/data.csv',index=False))


   product_id  quantity
0           1         2
   product_id  quantity
2           3         5
   product_id  quantity
1           2         3
   product_id  quantity
3           2         7


In [ ]:
rootdir = '/content/mydir'

dfs = []

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
      file_path = os.path.join(subdir, file)
      df = pd.read_csv(file_path)
      df['dt'] = file_path.split('/')[3]
      df['user_id'] = file_path.split('/')[4]
      dfs.append(df)

In [ ]:
pd.concat(dfs)

,product_id,quantity,dt,user_id
0,1,2,2020-12-24,1
0,3,5,2020-12-24,2
0,2,3,2020-12-23,1
0,2,7,2020-12-23,2
